# 通过LSTM网络改进Jazz Solo

欢迎来到本周的最后编程任务！ 在本笔记本中，您将实现一个使用LSTM生成音乐的模型。 您甚至可以在作业结束时听自己的音乐。

**您将学习：**
- 将LSTM应用于音乐生成。
- 通过深度学习生成自己的爵士音乐。

请运行以下单元格以加载此作业中所需的所有软件包。 这可能需要几分钟的时间。

In [ ]:
from __future__ import print_function
import IPython
import sys
from music21 import *
import numpy as np
from grammar import *
from qa import *
from preprocess import * 
from music_utils import *
from data_utils import *
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K

## 1 - 问题陈述

您想专门为朋友的生日创作爵士音乐。 但是，您不知道任何乐器或音乐作品。 幸运的是，您了解深度学习并且可以使用LSTM网络来解决此问题。

您将训练一个网络，以代表已表演作品的风格生成新颖的爵士独奏。

<img src="images/jazz.jpg" style="width:450;height:300px;">


### 1.1 - 数据集

您将在爵士音乐的语料库上训练算法。 运行下面的单元格，收听训练集中的音频片段：

In [ ]:
IPython.display.Audio('./data/30s_seq.mp3')

我们已经对音乐数据进行了预处理，以根据音乐“值”呈现音乐数据。您可以非正式地将每个“值”视为一个音符，其中包括一个音调和一个持续时间。例如，如果您按下特定的钢琴键0.5秒钟，则您刚刚演奏了一个音符。在音乐理论中，“值”实际上比这复杂得多-具体来说，它还捕获了同时演奏多个音符所需的信息。例如，演奏音乐作品时，您可以同时按下两个钢琴键（同时演奏多个音符会产生所谓的“和弦”）。但是我们无需担心音乐理论的细节。出于此分配的目的，您需要知道的是，我们将获取值的数据集，并将学习RNN模型以生成值序列。

我们的音乐生成系统将使用78个唯一值。运行以下代码以加载原始音乐数据并将其预处理为值。这可能需要几分钟。

In [ ]:
X, Y, n_values, indices_values = load_music_utils()
print('shape of X:', X.shape)
print('number of training examples:', X.shape[0])
print('Tx (length of sequence):', X.shape[1])
print('total # of unique values:', n_values)
print('Shape of Y:', Y.shape)

您刚刚加载了以下内容：

- `X`：这是一个(m, $T_x$, 78)维数组。我们有m个训练示例，每个训练示例都是$T_x =30$音乐值的摘要。在每个时间步长处，输入都是78个不同的可能值之一，表示为一个热向量。因此，例如，X[i,t,:]是一个热向量，表示在时间t处第i个示例的值。

- `Y`：本质上与`X`相同，但向左（过去）移了一步。与恐龙相似，我们对使用先前值预测下一个值的网络感兴趣，因此，给定x langle1时，我们的序列模型将尝试预测$y^{\langle t \rangle}$ 时，我们的序列模型将尝试预测 $x^{\langle 1\rangle}, \ldots, x^{\langle t \rangle}$。但是，将 `Y`中的数据重新排序为维度$(T_y, m, 78)$，其中$T_y = T_x$。这种格式使以后提供给LSTM更方便。

- `n_values`：该数据集中唯一值的数量。应该是78。

- `indices_values`：python字典，从0-77映射到音乐值。

### 1.2 - Overview of our model

这是我们将使用的模型的体系结构。 这与您在上一个笔记本中使用的Dinosaurus模型相似，不同之处在于您将在Keras中实现它。 架构如下：

<img src="images/music_generation.png" style="width:600;height:400px;">

<!--
<img src="images/djmodel.png" style="width:600;height:400px;">
<br>
<caption><center> **Figure 1**: LSTM model. $X = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$ is a window of size $T_x$ scanned over the musical corpus. Each $x^{\langle t \rangle}$ is an index corresponding to a value (ex: "A,0.250,< m2,P-4 >") while $\hat{y}$ is the prediction for the next value  </center></caption>
!--> 

我们将在更长的音乐片段中随机抽取30个值的片段来训练模型。 因此，我们不必费心设置第一个输入$x^{\langle 1 \rangle} = \vec{0}$，因为我们现在大部分代码段都用它来表示恐龙名称的开头。 音频开始于一段音乐的中间。 我们将每个片段设置为具有相同的长度$T_x = 30$ ，以使向量化更加容易。

## 2 - 建立模型

在这一部分中，您将构建和训练一个学习音乐模式的模型。 为此，您需要构建一个模型，该模型采用形状为$(T_x, m, 78)$的X和形状为$(m, T_y, 78)$的Y。 我们将使用具有64维隐藏状态的LSTM。 让我们设置 `n_a = 64`。


In [ ]:
n_a = 64 


这是创建具有多个输入和输出的Keras模型的方法。如果您正在构建RNN，即使在测试时间，整个输入序列$x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, \ldots, x^{\langle T_x \rangle}$都是*预先给定*，例如，如果输入是单词，而输出是标签，则Keras具有简单的内置函数来构建模型。但是，对于序列生成，在测试时我们并不预先知道$x^{\langle t\rangle}$的所有值；相反，我们使用$x^{\langle t\rangle} = y^{\langle t-1 \rangle}$一次生成一个。因此，代码会更加复杂，并且您将需要实现自己的for循环来迭代不同的时间步长。

函数`djmodel()`将使用for循环调用LSTM层$T_x$次，重要的是所有$T_x$副本具有相同的权重。也就是说，它不应该每次都重新初始化权重-$T_x$步骤应具有权重。在Keras中实现可共享权重的图层的关键步骤是：
1. 定义图层对象（为此，我们将使用全局变量）。
2. 在传播输入时调用这些对象。

我们已将所需的图层对象定义为全局变量。请运行下一个单元格以创建它们。请检查Keras文档以确保您了解这些层是什么：[Reshape()](https://keras.io/layers/core/#reshape)，[LSTM()](https://keras.io/layers/recurrent/#lstm), [Dense()](https://keras.io/layers/core/#dense)。

In [ ]:
reshapor = Reshape((1, 78))                        # Used in Step 2.B of djmodel(), below
LSTM_cell = LSTM(n_a, return_state = True)         # Used in Step 2.C
densor = Dense(n_values, activation='softmax')     # Used in Step 2.D

现在， `reshapor`，`LSTM_cell`和`densor`都是层对象，您可以使用它们来实现`djmodel()`。 为了通过这些层之一传播Keras张量对象X，请使用`layer_object(X)`（如果需要多个输入，则使用 `layer_object([X,Y])`）。 例如，`reshapor(X)`将通过上面定义的 `Reshape((1,78))`层传播X。

 
**练习**：实现`djmodel（）`。您将需要执行2个步骤：

1. 创建一个空列表“输出”以在每个时间步保存LSTM单元的输出。
2. 循环$t \in 1, \ldots, T_x$：

    A. 从X选择第“ t”个时间步长向量。此选择的形状应为（78，）。为此，请使用以下代码行在Keras中创建自定义 [Lambda](https://keras.io/layers/core/#lambda)层：
```    
           x = Lambda(lambda x: X[:,t,:])(X)
``` 
查看Keras文档以了解其作用。它正在创建一个“临时”或“未命名”函数（Lambda函数就是该函数），以提取适当的一键热矢量，并将该函数作为Keras的 `Layer` 对象应用于 `X`。

    B.将x重塑为（1,78）。您可能会发现`reshapor（）`层（在下面定义）很有用。

    C.运行x通过LSTM_cell的一个步骤。记住要使用上一步的隐藏状态$a$和单元状态$c$初始化LSTM_cell。使用以下格式：
```python
a, _, c = LSTM_cell(input_x, initial_state=[previous hidden state, previous cell state])
```

    D.使用`densor`通过密实+ softmax层传播LSTM的输出激活值。
    
    E.将预测值追加到“输出”列表中

In [ ]:
# GRADED FUNCTION: djmodel

def djmodel(Tx, n_a, n_values):
    """
    Implement the model
    
    Arguments:
    Tx -- length of the sequence in a corpus
    n_a -- the number of activations used in our model
    n_values -- number of unique values in the music data 
    
    Returns:
    model -- a keras model with the 
    """
    
    # Define the input of your model with a shape 
    X = Input(shape=(Tx, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    
    ### START CODE HERE ### 
    # Step 1: Create empty list to append the outputs while you iterate (≈1 line)
    outputs = None
    
    # Step 2: Loop
    for t in range(Tx):
        
        # Step 2.A: select the "t"th time step vector from X. 
        x = None
        # Step 2.B: Use reshapor to reshape x to be (1, n_values) (≈1 line)
        x = None
        # Step 2.C: Perform one step of the LSTM_cell
        a, _, c = None
        # Step 2.D: Apply densor to the hidden state output of LSTM_Cell
        out = None
        # Step 2.E: add the output to "outputs"
        None
        
    # Step 3: Create model instance
    model = None
    
    ### END CODE HERE ###
    
    return model

运行以下单元格以定义模型。 我们将使用`Tx=30`, `n_a=64` （LSTM激活的维数）和`n_values=78`。 该单元可能需要几秒钟才能运行。

In [ ]:
model = djmodel(Tx = 30 , n_a = 64, n_values = 78)

现在，您需要编译模型以进行训练。 我们将亚当和绝对交叉熵损失。

In [ ]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

最后，让LSTM的初始状态将`a0`和`c0`初始化为零。

In [ ]:
m = 60
a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))

现在让我们拟合模型！ 由于成本函数希望以这种格式提供 `Y`（每个时间步一个列表项），因此我们将 `Y`变成一个列表。 因此， `list(Y)`是一个包含30个项目的列表，其中每个列表项目的形状均为（60,78）。 让我们训练100个时代。 这将需要几分钟。

In [ ]:
model.fit([X, a0, c0], list(Y), epochs=100)

您应该看到模型损失减少。 现在您已经训练了一个模型，让我们继续最后一部分以实现推理算法并生成一些音乐！

## 3 - 生成音乐

您现在拥有一个受过训练的模型，该模型已经了解了爵士独奏者的模式。现在让我们使用此模型来合成新音乐。

#### 3.1 - 预测和采样

<img src="images/music_gen.png" style="width:600;height:400px;">

在采样的每个步骤中，您将以LSTM先前状态的激活`a`和单元状态`c` 作为输入，向前传播一步，并获得新的输出激活以及单元状态。然后，可以像以前一样使用`densor`来使用新的激活 `a` 来生成输出。

首先，我们将初始化 `x0`以及LSTM激活，并将像元值`a0`和`c0`初始化为零。


<！-
您将要构建一个函数来为您进行此推断。您的函数将采用您以前的模型以及要采样的时间步长`Ty`。它将返回一个可以为您生成序列的keras模型。此外，该功能包含`78`个单位的密集层和激活次数。
！->


**练习：**实施以下功能，以采样一系列音乐值。这是在生成TyTy输出字符的for循环中需要实现的一些关键步骤：

步骤2.A：使用`LSTM_Cell`，它输入上一步的`c`和`a`来生成当前步骤的`c`和`a`。

步骤2.B：使用`densor`（先前定义）在`a`上计算softmax，以获取当前步骤的输出。

步骤2.C：通过将刚刚生成的输出附加到`outputs`中来保存。

步骤2.D：将x采样为“出局”的单行本（预测），以便将其传递到下一个LSTM步骤。我们已经提供了使用 [Lambda](https://keras.io/layers/core/#lambda) 。
```python
x = Lambda(one_hot)(out) 
```
[次要技术说明：这行代码不是根据`out`中的概率随机采样一个值，而是使用argmax实际上在每一步选择一个最可能的注释。]

In [ ]:
# GRADED FUNCTION: music_inference_model

def music_inference_model(LSTM_cell, densor, n_values = 78, n_a = 64, Ty = 100):
    """
    Uses the trained "LSTM_cell" and "densor" from model() to generate a sequence of values.
    
    Arguments:
    LSTM_cell -- the trained "LSTM_cell" from model(), Keras layer object
    densor -- the trained "densor" from model(), Keras layer object
    n_values -- integer, umber of unique values
    n_a -- number of units in the LSTM_cell
    Ty -- integer, number of time steps to generate
    
    Returns:
    inference_model -- Keras model instance
    """
    
    # Define the input of your model with a shape 
    x0 = Input(shape=(1, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    x = x0

    ### START CODE HERE ###
    # Step 1: Create an empty list of "outputs" to later store your predicted values (≈1 line)
    outputs = None
    
    # Step 2: Loop over Ty and generate a value at every time step
    for t in range(None):
        
        # Step 2.A: Perform one step of LSTM_cell (≈1 line)
        a, _, c = None
        
        # Step 2.B: Apply Dense layer to the hidden state output of the LSTM_cell (≈1 line)
        out = None

        # Step 2.C: Append the prediction "out" to "outputs". out.shape = (None, 78) (≈1 line)
        None
        
        # Step 2.D: Select the next value according to "out", and set "x" to be the one-hot representation of the
        #           selected value, which will be passed as the input to LSTM_cell on the next step. We have provided 
        #           the line of code you need to do this. 
        x = None
        
    # Step 3: Create model instance with the correct "inputs" and "outputs" (≈1 line)
    inference_model = None
    
    ### END CODE HERE ###
    
    return inference_model

运行下面的单元格以定义您的推理模型。 该模型经过硬编码以生成50个值。

In [ ]:
inference_model = music_inference_model(LSTM_cell, densor, n_values = 78, n_a = 64, Ty = 50)

最后，这将创建零值向量，用于初始化`x`和LSTM状态变量`a`和`c`。

In [ ]:
x_initializer = np.zeros((1, 1, 78))
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

**练习**：实现`predict_and_sample（）`。 此函数接受许多参数，包括输入[x_initializer, a_initializer, c_initializer].。 为了预测与此输入对应的输出，您将需要执行3个步骤：
1.根据您的输入集，使用推理模型预测输出。 输出`pred`应该是长度为20的列表，其中每个元素都是一个形状为($T_y$, n_values)的numpy数组。
2.将`pred`转换为$T_y$索引的numpy数组。 通过使用`pred`列表中元素的`argmax`来计算每个对应的索引。 [Hint](https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmax.html)。
3.将索引转换为一键向量表示。 [Hint](https://keras.io/utils/#to_categorical)。

In [ ]:
# GRADED FUNCTION: predict_and_sample

def predict_and_sample(inference_model, x_initializer = x_initializer, a_initializer = a_initializer, 
                       c_initializer = c_initializer):
    """
    Predicts the next value of values using the inference model.
    
    Arguments:
    inference_model -- Keras model instance for inference time
    x_initializer -- numpy array of shape (1, 1, 78), one-hot vector initializing the values generation
    a_initializer -- numpy array of shape (1, n_a), initializing the hidden state of the LSTM_cell
    c_initializer -- numpy array of shape (1, n_a), initializing the cell state of the LSTM_cel
    
    Returns:
    results -- numpy-array of shape (Ty, 78), matrix of one-hot vectors representing the values generated
    indices -- numpy-array of shape (Ty, 1), matrix of indices representing the values generated
    """
    
    ### START CODE HERE ###
    # Step 1: Use your inference model to predict an output sequence given x_initializer, a_initializer and c_initializer.
    pred = None
    # Step 2: Convert "pred" into an np.array() of indices with the maximum probabilities
    indices = None
    # Step 3: Convert indices to one-hot vectors, the shape of the results should be (1, )
    results = None
    ### END CODE HERE ###
    
    return results, indices

In [ ]:
results, indices = predict_and_sample(inference_model, x_initializer, a_initializer, c_initializer)
print("np.argmax(results[12]) =", np.argmax(results[12]))
print("np.argmax(results[17]) =", np.argmax(results[17]))
print("list(indices[12:18]) =", list(indices[12:18]))

**预期输出**: 您的结果可能会有所不同，因为Keras的结果并非完全可预测。 但是，如果您如上所述用model.fit（）将LSTM_cell训练了正好100个纪元，那么您很有可能会观察到一系列不完全相同的索引。 此外，您应该注意：np.argmax（results[12]）是list（indices[12:18]）的第一个元素，np.argmax（results[17]）是list（indices[12:18]）的最后一个元素 。
<table>
    <tr>
        <td>
            **np.argmax(results[12])** =
        </td>
        <td>
        1
        </td>
    </tr>
    <tr>
        <td>
            **np.argmax(results[12])** =
        </td>
        <td>
        42
        </td>
    </tr>
    <tr>
        <td>
            **list(indices[12:18])** =
        </td>
        <td>
            [array([1]), array([42]), array([54]), array([17]), array([1]), array([42])]
        </td>
    </tr>
</table>

#### 3.3 - 生成音乐

最后，您准备好生成音乐了。您的RNN会生成一个值序列。以下代码通过首先调用您的`predict_and_sample（）`函数来生成音乐。然后将这些值后期处理成和弦（意味着可以同时演奏多个值或音符）。

大多数计算音乐算法都使用某些后处理，因为没有这种后处理很难生成听起来不错的音乐。后处理通过确保同一声音不会重复太多，两个连续的音符彼此之间的音高相距不远等来清理生成的音频。有人可能会说，其中许多后处理步骤都是黑客。同样，许多音乐产生文学也集中于手工制作后处理器，并且许多输出质量取决于后处理的质量，而不仅仅是RNN的质量。但是这种后处理的确有很大的不同，因此也请在我们的实现中使用它。

让我们做些音乐吧！

运行以下单元以生成音乐并将其记录到您的`out_stream`中。 这可能需要几分钟。

In [ ]:
out_stream = generate_music(inference_model)

要听音乐，请单击文件->打开...，然后转到“输出/”并下载“ my_music.midi”。 您可以使用可以读取Midi文件的应用程序在计算机上播放该文件，也可以使用免费的在线“ MIDI to mp3”转换工具之一将其转换为mp3。

作为参考，这也是我们使用此算法生成的30秒音频剪辑。

In [ ]:
IPython.display.Audio('./data/30s_trained_model.mp3')

### 恭喜！

您已经到了笔记本的结尾。

<font color="blue">
这是您应该记住的：
    
- 序列模型可用于生成音乐值，然后将其后处理为Midi音乐。
- 可以使用非常相似的模型来生成恐龙名称或生成音乐，主要区别是输入模型的输入。
- 在Keras中，序列生成涉及定义具有共享权重的图层，然后在不同的时间步长$1, \ldots, T_x$中重复这些步骤。

恭喜您完成了这项任务并生成了爵士独奏！

**参考**

本笔记本中提出的想法主要来自以下引用的三篇计算音乐论文。 这里的实现也获得了很大的启发，并使用了Ji-Sung Kim的github存储库中的许多组件。

- Ji-Sung Kim, 2016, [deepjazz](https://github.com/jisungk/deepjazz)
- Jon Gillick, Kevin Tang and Robert Keller, 2009. [Learning Jazz Grammars](http://ai.stanford.edu/~kdtang/papers/smc09-jazzgrammar.pdf)
- Robert Keller and David Morrison, 2007, [A Grammatical Approach to Automatic Improvisation](http://smc07.uoa.gr/SMC07%20Proceedings/SMC07%20Paper%2055.pdf)
- François Pachet, 1999, [Surprising Harmonies](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.5.7473&rep=rep1&type=pdf)

我们也感谢FrançoisGermain的宝贵反馈。